In [1]:
import os
import io
import re
import oeis
import functools
import torch

path = '/Users/nolte/Dropbox/ML'

cache_path = "./data"
os.makedirs(cache_path, exist_ok=True)

def readESymb(loop, path, file):
    name = os.path.join(path, file)
    assert os.path.isfile(name), name
    res = ''
    prefix = 'Esymb'
    with open(name, 'rt') as f:
        reading_form = False
        for line in f:
            if not reading_form:
                if not line.startswith(prefix + '[' + str(loop) + ']'): continue
                res = ''
                reading_form = True
            res += line[:-2] if line[-2] == '\\' else line[:-1]
            if line[-2] in [":", ";"]:
                break
    return res


def convert(loop):
    file = 'EZ6_symb_new_norm' if loop == 6 else 'EZ_symb_new_norm'
    s = re.split(":=|SB\(|\)", re.sub('[,*]', '', readESymb(loop, path, file)))[1:-1]
    keys = s[1::2]
    values = [int(re.sub('[+-]$',t[0]+'1',t)) for t in s[0::2]]
    return {k:v for k,v in zip(keys,values)}



def query_key(key,verbose=False):
    l = len(key)
    if l==0:
        if verbose: print('empty key')
        return 0
    if l%2==1:
        if verbose: print("odd length key")
        return 0
    if key[0] in ['d', 'e', 'f'] or key[-1] in ['a', 'b', 'c']:
        if verbose: print("stupid zero: begin/end")
        return 0
    for i in range(l-1):
        if (key[i] == 'a' and key[i+1] == 'd') \
            or (key[i] == 'b' and key[i+1] == 'e') \
            or (key[i] == 'c' and key[i+1] == 'f'):
                if verbose: print('stupid zero: adjacency')
                return 0
        if (key[i] == 'd' and key[i+1] in ['a','e','f'])\
            or (key[i] == 'e' and key[i+1] in ['b','d','f'])\
            or (key[i] == 'f' and key[i+1] in ['c','d','e']):
                if verbose: print('stupid zero: adjacency')
                return 0


    loop  = l//2
    if loop < 1 or loop > 6:
        print("loop:", loop, "1 to 6 only:")
        return 0
    if key in data[loop].keys():
        return data[loop][key]
    return 0



data = [[]]
print("loading...")
for i in range(1,7):
    e=convert(i)
    data.append(e)
print("loaded")

for i in range(7):
    print(i, len(data[i]))


loading...
loaded
0 0
1 6
2 12
3 636
4 11208
5 263880
6 4916466


In [5]:
def tokenize(word, max_length=12):
  result = [ord(c) - ord('a') + 1 for c in word]
  result += [0] * (max_length - len(result))
  return torch.tensor(result).long()

all_data = [
  (tokenize(word), torch.tensor([val]))
  for i in range(1, 7)
  for word, val in data[i].items()
]


In [12]:
X, Y = torch.stack([x[0] for x in all_data]), torch.stack([x[1] for x in all_data])
torch.save((X, Y), os.path.join(cache_path, "data.pt"))
